In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [3]:
ALL_EVALUATIONS="tests/evaluation_09_retrievers_000_009_sampled_50_queries_gpt-4-1106-preview_20240103.tsv"

BM25_RUN="tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt"
BM25_E5_RUN="tests/bm25_e5_docs_000_999_50_sampled_queries_run.txt"
BM25_MT5_RUN="tests/200-queries-passagens-by-BM25-mT5_v4_with_query_id_run.txt"
SPLADE_RUN="tests/splade_pt_result_with_clueweb_id_run.txt"
COLBERTX_RUN="tests/colbertx_all_znormalized_validation_scores_000_199_run.txt"
E5_RUN="tests/e5_docs_000_009_50_sampled_queries_score_inv_run.txt"
E5_BASE_RUN="tests/e5_base_docs_000_009_50_sampled_queries_score_inv_run.txt"
BM25_1M_RUN="tests/bm25_1M_docs_000_009_sampled_50_queries_run.txt"

In [4]:
BM25_1M_MASSIVEWEB="tests/bm25_massiveweb_1M_docs_000_009_sampled_50_queries.tsv"

In [5]:
retrievers_results = []

### Format GPT-4 evaluations to qrels format

In [6]:
eval_df = pd.read_csv(ALL_EVALUATIONS, sep='\t')

In [7]:
eval_df

query_id                                                  query  \
0            1       Qual a maior caracter√≠stica da fauna brasileira?   
1            1       Qual a maior caracter√≠stica da fauna brasileira?   
2            1       Qual a maior caracter√≠stica da fauna brasileira?   
3            1       Qual a maior caracter√≠stica da fauna brasileira?   
4            1       Qual a maior caracter√≠stica da fauna brasileira?   
...        ...                                                    ...   
2956       199  Quais foram os principais artistas do Impressionismo?   
2957       199  Quais foram os principais artistas do Impressionismo?   
2958       199  Quais foram os principais artistas do Impressionismo?   
2959       199  Quais foram os principais artistas do Impressionismo?   
2960       199  Quais foram os principais artistas do Impressionismo?   

                        passage_id    mT5  splade  colbertx  bm25_e5   bm25  \
0      clueweb22-pt0000-82-13187_0  False   False      True    False  False   
1      clueweb22-pt0000-59-04599_2  False   False      True    False  False   
2     clueweb22-pt0000-64-06285_39   True    True     False    False  False   
3      clueweb22-pt0000-77-06884_1  False    True      True    False  False   
4      clueweb22-pt0000-78-09747_0   True   False     False    False   True   
...                            ...    ...     ...       ...      ...    ...   
2956   clueweb22-pt0000-48-17441_2  False   False     False    False  False   
2957   clueweb22-pt0002-04-01418_0  False   False     False    False  False   
2958   clueweb22-pt0001-24-17228_0  False   False     False    False  False   
2959   clueweb22-pt0000-08-07683_2  False   False     False    False  False   
2960   clueweb22-pt0000-86-08729_0  False   False     False    False  False   

         e5  e5_base  bm25_1M  bm25_1M_massiveweb  \
0     False    False    False               False   
1     False    False    False               False   
2     False     True    False               False   
3     False    False    False               False   
4     False    False    False                True   
...     ...      ...      ...                 ...   
2956  False    False    False                True   
2957  False    False    False                True   
2958  False    False    False                True   
2959  False    False    False                True   
2960  False    False    False                True   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    passage  \
0                                                                                                                                                                                                                                                  

In [8]:
eval_df.drop_duplicates("query_id")[['query_id', 'query']]

query_id  \
0            1   
60           2   
130          9   
191         11   
245         13   
296         15   
339         17   
390         20   
463         21   
533         22   
587         24   
657         26   
713         28   
780         47   
854         49   
912         51   
960         54   
1017        60   
1066        62   
1137        64   
1190        68   
1264        84   
1319        95   
1389        98   
1452       105   
1516       113   
1566       115   
1626       117   
1699       126   
1776       127   
1811       128   
1859       136   
1920       147   
1980       152   
2041       153   
2106       154   
2153       160   
2203       161   
2249       163   
2302       167   
2375       170   
2439       172   
2489       180   
2541       181   
2611       182   
2665       189   
2726       193   
2785       195   
2835       196   
2902       199   

                                                                                           query  
0                                               Qual a maior caracter√≠stica da fauna brasileira?  
60                              Por que os pa√≠ses Guiana e Suriname n√£o s√£o filiados a Conmebol?  
130                                           quais pa√≠ses europeus seguem o regime monarquista?  
191      quais os crit√©rios de defini√ß√£o dos monumentos intitulados maravilhas do mundo moderno?  
245                                 Como o Brasil reagiu a epidemia de AIDS no fim do s√©culo XX?  
296                                                   Qual a maior torcida de futebol do Brasil?  
339           Quando se realizou o plebiscito popular para definir o sistema pol√≠tico do Brasil?  
390                                            Por que a legisla√ß√£o de um pa√≠s √© t√£o importante?  
463                                                       Qual o nosso atual sistema de governo?  
533                                                  Qual o sistema de governo da √Åfrica do Sul?  
587                                                          Por que dividir um pa√≠s em estados?  
657                                                Como podemos classificar o relevo brasileiro?  
713                                                Existem vantagens ao definir uma moeda √∫nica?  
780                                     Como transformar uma cidade pacata em um polo tur√≠stico?  
854                                         Quais s√£o os melhores parques nacionais de Portugal?  
912                                                  Qual a melhor cidade para ficar no Jalap√£o?  
960                                                               Quais s√£o os biomas do Brasil?  
1017                           Quando foi criada a consolida√ß√£o das leis trabalhistas no brasil?  
1066                                Quais partidos j√° ocuparam o cargo da presid√™ncia do Brasil?  
1137                           Por que os anos de 80 s√£o considerados a d√©cada perdida no Brasil  
1190                                   Quais s√£o os principais int√©rpretes da m√∫sica brasileira?  
1264                                           Qual √© a principal cidade do basquete brasileiro?  
1319                                        √â poss√≠vel conhecer o Pico da Bandeira com crian√ßas?  
1389                                   Qual o crit√©rio para classifica√ß√£o para a Copa do Brasil?  
1452                                                Onde est√° localizada a Pra√ßa XV de Novembro?  
1516                                                        Fatores de risco para cancer de pele  
1566                                                 Cuidados ao usar materiais perfurocortantes  
1626                                                                   Delirio versus alucina√ß√£o  
1699                                     Quantas placas s√£o formadas pelos materiais condutores?  
1776                                              O que √© a associa√ß√£o 

#### Check scores distribution on the current qrels

In [9]:
np.unique(eval_df['score'].to_numpy(), return_counts=True)

(array([0, 1, 2, 3]), array([1421,  591,  453,  496]))

In [10]:
for group_name, group_df in eval_df.groupby('query_id'):
    print("\nquery_id={}".format(group_name))
    
    print(np.unique(group_df['score'].to_numpy(), return_counts=True))


query_id=1
(array([0, 1, 2, 3]), array([ 9, 32, 12,  7]))

query_id=2
(array([0, 1, 2, 3]), array([65,  2,  1,  2]))

query_id=9
(array([0, 1, 2, 3]), array([42,  5,  9,  5]))

query_id=11
(array([0, 1, 2, 3]), array([32, 12,  8,  2]))

query_id=13
(array([0, 1, 2, 3]), array([25, 10, 12,  4]))

query_id=15
(array([0, 1, 2, 3]), array([16, 13,  1, 13]))

query_id=17
(array([0, 1, 2, 3]), array([15,  7,  1, 28]))

query_id=20
(array([0, 1, 2, 3]), array([13, 38, 18,  4]))

query_id=21
(array([0, 1, 2, 3]), array([37, 17,  2, 14]))

query_id=22
(array([0, 1, 3]), array([39, 10,  5]))

query_id=24
(array([0, 1, 2, 3]), array([42, 17,  9,  2]))

query_id=26
(array([0, 1, 2, 3]), array([13,  8,  9, 26]))

query_id=28
(array([0, 1, 2, 3]), array([40, 10, 13,  4]))

query_id=47
(array([0, 1, 2]), array([35, 23, 16]))

query_id=49
(array([0, 1, 2, 3]), array([20, 19, 18,  1]))

query_id=51
(array([0, 1, 2, 3]), array([21, 16, 10,  1]))

query_id=54
(array([0, 1, 2, 3]), array([ 1, 12, 14, 30]

In [11]:
eval_df.groupby('query_id').count()

query  passage_id  mT5  splade  colbertx  bm25_e5  bm25  e5  \
query_id                                                                
1            60          60   60      60        60       60    60  60   
2            70          70   70      70        70       70    70  70   
9            61          61   61      61        61       61    61  61   
11           54          54   54      54        54       54    54  54   
13           51          51   51      51        51       51    51  51   
15           43          43   43      43        43       43    43  43   
17           51          51   51      51        51       51    51  51   
20           73          73   73      73        73       73    73  73   
21           70          70   70      70        70       70    70  70   
22           54          54   54      54        54       54    54  54   
24           70          70   70      70        70       70    70  70   
26           56          56   56      56        56       56    56  56   
28           67          67   67      67        67       67    67  67   
47           74          74   74      74        74       74    74  74   
49           58          58   58      58        58       58    58  58   
51           48          48   48      48        48       48    48  48   
54           57          57   57      57        57       57    57  57   
60           49          49   49      49        49       49    49  49   
62           71          71   71      71        71       71    71  71   
64           53          53   53      53        53       53    53  53   
68           74          74   74      74        74       74    74  74   
84           55          55   55      55        55       55    55  55   
95           70          70   70      70        70       70    70  70   
98           63          63   63      63        63       63    63  63   
105          64          64   64      64        64       64    64  64   
113          50          50   50      50        50       50    50  50   
115          60          60   60      60        60       60    60  60   
117          73          73   73      73        73       73    73  73   
126          77          77   77      77        77       77    77  77   
127          35          35   35      35        35       35    35  35   
128          48          48   48      48        48       48    48  48   
136          61          61   61      61        61       61    61  61   
147          60          60   60      60        60       60    60  60   
152          61          61   61      61        61       61    61  61   
153          65          65   65      65        65       65    65  65   
154          47          47   47      47        47       47    47  47   
160          50          50   50      50        50       50    50  50   
161          46          46   46      46        46       46    46  46   
163          53          53   53      53        53       53    53  53   
167          73          73   73      73        73       73    73  73   
170          64          64   64      64        64       64    64  64   
172          50          50   50      50        50       50    50  50   
180          52          52   52      52        52       52    52  52   
181          70          70   70      70        70       70    70  70   
182          54          54   54      54        54       54    54  54   
189          61          61   61      61        61       61    61  61   
193          59          59   59      59        59       59    59  59   
195          50          50   50      50        50       50    50  50   
196          67          67   67      67        67       67    67  67   
199          59          59   59      59        59       59    59  59   

          e5_base  bm25_1M  bm25_1M_massiveweb  passage  score  reasoning  \
query_id                                                                    
1              60       60                  60       60    

In [12]:
eval_df.groupby('query_id').count()['query'].describe()

count    50.000000
mean     59.220000
std       9.607161
min      35.000000
25%      51.250000
50%      59.500000
75%      67.000000
max      77.000000
Name: query, dtype: float64

#### Prepare the qrels format for trec_eval tools

In [13]:
eval_df['0'] = '0'

In [14]:
qrels_filename = os.path.join(os.path.dirname(ALL_EVALUATIONS), "{}_qrel.txt".format(os.path.splitext(os.path.basename(ALL_EVALUATIONS))[0]))

In [15]:
qrels_filename

'tests/evaluation_09_retrievers_000_009_sampled_50_queries_gpt-4-1106-preview_20240103_qrel.txt'

In [16]:
eval_df.sort_values('query_id')[['query_id', '0', 'passage_id', 'score']].to_csv(qrels_filename, header=None, index=False, sep=" ")

### Compute BM25 nDCG@10

In [17]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt

In [18]:
retrievers_results.append({'retriever':'BM25', 'nDCG@10': result[0].split('\t')[-1]})

In [19]:
retrievers_results

[{'retriever': 'BM25', 'nDCG@10': '0.4580'}]

### Compute BM25 + E5 nDCG@10

In [20]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {BM25_E5_RUN}

In [21]:
retrievers_results.append({'retriever':'BM25+E5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 + mT5 nDCG@10

In [22]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {BM25_MT5_RUN}

In [23]:
retrievers_results.append({'retriever':'BM25+mT5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute SPLADE nDCG@10

In [24]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {SPLADE_RUN}

In [25]:
retrievers_results.append({'retriever':'SPLADE_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERTX nDCG@10

In [26]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {COLBERTX_RUN}

In [27]:
retrievers_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 nDCG@10

In [28]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {E5_RUN}

In [29]:
retrievers_results.append({'retriever':'E5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 base nDCG@10

In [30]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {E5_BASE_RUN}

In [31]:
retrievers_results.append({'retriever':'E5 base', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 1M nDCG@10

In [32]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {BM25_1M_RUN}

retrievers_results.append({'retriever':'BM25 1M', 'nDCG@10': result[0].split('\t')[-1]})

### Prepare BM25 1M massiveweb filtering retrieval results in run format

In [33]:
bm25_1M_df = pd.read_csv(BM25_1M_MASSIVEWEB, sep="\t")

In [34]:
bm25_1M_df.groupby("query_id").head(10).count()

query_id       500
Q0             500
colbertx_id    500
position       500
score          500
comment        500
passage_id     500
query          500
passage        500
dtype: int64

In [35]:
bm25_1M_df

query_id  Q0  colbertx_id  position      score   comment  \
0           1  Q0       264425         1  11.670300  Anserini   
1           1  Q0      3718313         2  10.147600  Anserini   
2           1  Q0      4178525         3   9.934500  Anserini   
3           1  Q0      3299432         4   9.916500  Anserini   
4           1  Q0      2605802         5   9.785500  Anserini   
5           1  Q0      8464958         6   9.689000  Anserini   
6           1  Q0      6839226         7   9.662400  Anserini   
7           1  Q0       826610         8   9.243300  Anserini   
8           1  Q0      2351613         9   9.238700  Anserini   
9           1  Q0      2605803        10   9.127000  Anserini   
10          2  Q0      9973433         1  16.865499  Anserini   
11          2  Q0      2435657         2  16.846100  Anserini   
12          2  Q0      3951208         3  15.522000  Anserini   
13          2  Q0      5316825         4  15.277800  Anserini   
14          2  Q0      1199265         5  14.902700  Anserini   
15          2  Q0      5422544         6  14.529400  Anserini   
16          2  Q0      1801730         7  14.135600  Anserini   
17          2  Q0      5775299         8  13.740400  Anserini   
18          2  Q0      2413931         9  13.696600  Anserini   
19          2  Q0      7511414        10  13.548600  Anserini   
20          9  Q0      9150842         1  11.825800  Anserini   
21          9  Q0      2865063         2  11.572100  Anserini   
22          9  Q0      7888308         3  10.905000  Anserini   
23          9  Q0      9537032         4  10.745800  Anserini   
24          9  Q0      7845754         5  10.659700  Anserini   
25          9  Q0      4892046         6  10.037600  Anserini   
26          9  Q0      5243938         7   9.992300  Anserini   
27          9  Q0      2389179         8   9.852200  Anserini   
28          9  Q0      5201160         9   9.568200  Anserini   
29          9  Q0      6718816        10   9.528400  Anserini   
30         11  Q0      3419339         1  16.433201  Anserini   
31         11  Q0      8845068         2  14.652700  Anserini   
32         11  Q0      1894320         3  14.360500  Anserini   
33         11  Q0      1701354         4  13.336200  Anserini   
34         11  Q0      1894322         5  12.602800  Anserini   
35         11  Q0      8301161         6  12.511900  Anserini   
36         11  Q0      2171340         7  11.572900  Anserini   
37         11  Q0      2343937         8  11.532900  Anserini   
38         11  Q0      9577450         9  11.464300  Anserini   
39         11  Q0      1475171        10  11.457000  Anserini   
40         13  Q0      3512602         1  15.141600  Anserini   
41         13  Q0      5053329         2  14.789200  Anserini   
42         13  Q0      5906027         3  12.961200  Anserini   
43         13  Q0      4819699         4  12.847300  Anserini   
44         13  Q0       614269         5  12.790500  Anserini   
45         13  Q0      8432326         6  12.771000  Anserini   
46         13  Q0      2990034         7  12.695600  Anserini   
47         13  Q0      6656231         8  12.641700  Anserini   
48         13  Q0       725173         9  12.601200  Anserini   
49         13  Q0      7911563        10  12.468200  Anserini   
50         15  Q0      1814925         1  11.852000  Anserini   
51         15  Q0      7896696         2  11.830400  Anserini   
52         15  Q0      9635813         3  11.374300  Anserini   
53         15  Q0      6104336         4  11.139600  Anserini   
54         15  Q0      4963350         5  11.064900  Anserini   
55         15  Q0      6836305         6  10.955800  Anserini   
56         15  Q0      8588047         7  10.887500  Anserini   
57         15  Q0      1500884         8  10.871900  Anserini   
58         15  Q0      2343297         9  10.758800  Anserini   
59         15  Q0      5396454        10  10.749100  Anserini   
60         17  Q0      7874391         1

In [36]:
bm25_1M_run_df = bm25_1M_df.sort_values(["query_id", "score"], ascending=[True, False]).groupby("query_id").head(10)

In [37]:
bm25_1M_run_df.shape

(500, 9)

In [38]:
bm25_1M_run_df['Q0'] = "Q0"
bm25_1M_run_df['comment'] = "gpt-4-1106-preview"
bm25_1M_run_df['index'] = np.tile(list(range(1, 11)), 50)

In [39]:
bm25_1M_run_df.head(12)

query_id  Q0  colbertx_id  position      score             comment  \
0          1  Q0       264425         1  11.670300  gpt-4-1106-preview   
1          1  Q0      3718313         2  10.147600  gpt-4-1106-preview   
2          1  Q0      4178525         3   9.934500  gpt-4-1106-preview   
3          1  Q0      3299432         4   9.916500  gpt-4-1106-preview   
4          1  Q0      2605802         5   9.785500  gpt-4-1106-preview   
5          1  Q0      8464958         6   9.689000  gpt-4-1106-preview   
6          1  Q0      6839226         7   9.662400  gpt-4-1106-preview   
7          1  Q0       826610         8   9.243300  gpt-4-1106-preview   
8          1  Q0      2351613         9   9.238700  gpt-4-1106-preview   
9          1  Q0      2605803        10   9.127000  gpt-4-1106-preview   
10         2  Q0      9973433         1  16.865499  gpt-4-1106-preview   
11         2  Q0      2435657         2  16.846100  gpt-4-1106-preview   

                      passage_id  \
0    clueweb22-pt0000-05-02747_0   
1    clueweb22-pt0000-78-09747_0   
2    clueweb22-pt0000-87-14633_2   
3    clueweb22-pt0000-69-05954_7   
4    clueweb22-pt0000-55-02859_0   
5    clueweb22-pt0001-77-01947_0   
6   clueweb22-pt0001-42-08363_12   
7    clueweb22-pt0000-17-01578_0   
8    clueweb22-pt0000-49-09984_1   
9    clueweb22-pt0000-55-02859_3   
10   clueweb22-pt0002-38-00517_0   
11   clueweb22-pt0000-51-04359_0   

                                                               query  \
0                   Qual a maior caracter√≠stica da fauna brasileira?   
1                   Qual a maior caracter√≠stica da fauna brasileira?   
2                   Qual a maior caracter√≠stica da fauna brasileira?   
3                   Qual a maior caracter√≠stica da fauna brasileira?   
4                   Qual a maior caracter√≠stica da fauna brasileira?   
5                   Qual a maior caracter√≠stica da fauna brasileira?   
6                   Qual a maior caracter√≠stica da fauna brasileira?   
7                   Qual a maior caracter√≠stica da fauna brasileira?   
8                   Qual a maior caracter√≠stica da fauna brasileira?   
9                   Qual a maior caracter√≠stica da fauna brasileira?   
10  Por que os pa√≠ses Guiana e Suriname n√£o s√£o filiados a Conmebol?   
11  Por que os pa√≠ses Guiana e Suriname n√£o s√£o filiados a Conmebol?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   passage  \
0                                                                                                                                                                                                                                                                                                                    

In [40]:
bm25_1M_run_df[['query_id', 'Q0', "passage_id", "index", "score", "comment"]].to_csv("tests/bm25_massivewevb_1M_docs_000_009_sampled_50_queries_run.txt", 
                                                                                     header=None, index=False, sep=" ")

#### Compute BM25 1M NDCG@10

In [41]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} "tests/bm25_massivewevb_1M_docs_000_009_sampled_50_queries_run.txt"

In [42]:
retrievers_results.append({'retriever':'BM25 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Check the overall results

In [43]:
pd.DataFrame(retrievers_results)

retriever nDCG@10
0                    BM25  0.4580
1                 BM25+E5  0.6541
2                BM25+mT5  0.7317
3            SPLADE_pt-BR  0.5948
4  ColBERT-X_mMARCO_pt-BR  0.6458
5                      E5  0.5729
6                 E5 base  0.4110
7                 BM25 1M  0.3254
8      BM25 1M massiveweb  0.3232